<a href="https://colab.research.google.com/github/Lucifer0190/Siamese/blob/main/MNIST_Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying Siamese Neural Network on MNIST Handwritten Digits Dataset

In [1]:
import numpy as np
import random
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.models as Models
import tensorflow.keras.losses as Losses
import tensorflow.keras.optimizers as Optimizers
import tensorflow.keras.activations as Activations
import tensorflow.keras.backend as K

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [3]:
classes = 10
input_shape = x_train.shape[1:]
epochs = 5

In [4]:
def euclidean_distance(vectors):
  x, y = vectors
  sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

## Custom Loss functions

In [5]:
def contrastive_loss(true_label,pred_label):
  margin = 1
  square_pred = K.square(pred_label)
  margin_square = K.square(K.maximum(margin - pred_label, 0))
  return K.mean(true_label * square_pred + (1 - true_label) * margin_square)

## Pairs creation for providing input in the network

In [6]:
def create_pairs(x, digit_indices):
  pairs = []
  labels = []
  n = min([len(digit_indices[d]) for d in range(classes)]) - 1
  for d in range(classes):
      for i in range(n):
          z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
          pairs += [[x[z1], x[z2]]]
          inc = random.randrange(1, classes)
          dn = (d + inc) % classes
          z1, z2 = digit_indices[d][i], digit_indices[dn][i]
          pairs += [[x[z1], x[z2]]]
          labels += [1.0, 0.0]
  return np.array(pairs), np.array(labels)

## Accuracy Calculation

In [7]:
def metrics_accuracy(true_label,pred_label):
  return K.mean(K.equal(true_label,K.cast(pred_label<0.5,true_label.dtype)))

In [8]:
def compute_accuracy(true_label,pred_label):
  pred = pred_label.ravel() < 0.5
  return np.mean(pred == true_label)

## Model Creation

In [9]:
def create_backbone(input_shape):
  input = Layers.Input(shape=input_shape)
  x = Layers.Flatten()(input)
  x = Layers.Dense(512,activation=Activations.relu)(x)
  x = Layers.Dropout(0.1)(x)
  x = Layers.Dense(128,activation=Activations.relu)(x)
  x = Layers.Dropout(0.1)(x)
  x = Layers.Dense(128,activation=Activations.relu)(x)
  return Models.Model(input,x)

In [10]:
backbone = create_backbone(input_shape)

input_a = Layers.Input(shape=input_shape)
input_b = Layers.Input(shape=input_shape)

features_a = backbone(input_a)
features_b = backbone(input_b)

In [11]:
distance = Layers.Lambda(euclidean_distance,output_shape = eucl_dist_output_shape)([features_a,features_b])

In [12]:
model = Models.Model([input_a,input_b], distance)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       (None, 128)          484096      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           functional_1[0][0]    

In [13]:
digit_indices = [np.where(y_train == i)[0] for i in range(classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

## Training

In [14]:
model.compile(loss=contrastive_loss,
              optimizer=Optimizers.RMSprop(),
              metrics=[metrics_accuracy])

In [15]:
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

Epoch 1/5
847/847 [==============================] - 17s 20ms/step - loss: 0.0800 - metrics_accuracy: 0.9097 - val_loss: 0.0335 - val_metrics_accuracy: 0.9657
Epoch 2/5
847/847 [==============================] - 16s 19ms/step - loss: 0.0292 - metrics_accuracy: 0.9709 - val_loss: 0.0263 - val_metrics_accuracy: 0.9717
Epoch 3/5
847/847 [==============================] - 17s 20ms/step - loss: 0.0199 - metrics_accuracy: 0.9792 - val_loss: 0.0215 - val_metrics_accuracy: 0.9759
Epoch 4/5
847/847 [==============================] - 17s 20ms/step - loss: 0.0152 - metrics_accuracy: 0.9840 - val_loss: 0.0233 - val_metrics_accuracy: 0.9720
Epoch 5/5
847/847 [==============================] - 17s 20ms/step - loss: 0.0128 - metrics_accuracy: 0.9865 - val_loss: 0.0208 - val_metrics_accuracy: 0.9767


## Compute final Accuracy

In [16]:
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

In [17]:
print('Training Accuracy: %0.2f%%' % (100 * tr_acc))
print('Testing Accuracy: %0.2f%%' % (100 * te_acc))

Training Accuracy: 99.15%
Testing Accuracy: 97.65%
